## Lista 5 - Aprendizagem de Máquina Probabilístico
- Aluno: Lucas Rodrigues Aragão - Graduação 538390

In [ ]:
import numpy as np
import pandas as pd
import scipy as sp
import jax

## Modelos

### Inferência com modelos de GP para regressão 
- Estimação 

    1. Inicializar hiperparâmetros $\theta = \big[  \sigma^2_f, w^2_1, \cdots, w^2_D, \sigma^2_y \big]^T$

    2. Repetir até convergir ou número de épocas

        1. Calcular a evidência do modelo $\log p(y|X, \theta)$.
        2. Calcular os gradientes analíticos $\frac{\partial \log p(y|X, \theta)}{\partial \theta}$.
        3. Atualizar $\theta\$ a partir dos gradientes.

    3. Retornar os hiperparâmetros

- Predição

1. Dado um novo padrão $x_\ast$ retornar a distribuição preditiva 

$$p(y| x_\ast, y, X, \hat{\theta}) = \mathcal{N}(y_\ast| \mu_\ast, \sigma^2_\ast + \sigma^2_y)$$

$$\mu_\ast = k_{f \ast}^T (K + \sigma^2_y I)^{-1} y$$

$$\sigma^2_\ast = k_{\ast \ast} - k_{f \ast}^T (K + \sigma^2_y I)^{-1} k_{f \ast}$$

Em que, $k_{f \ast} = [k(x_\ast, x_1), \cdots , k(x_\ast, x_N)]$ e $k_{\ast \ast} = k(x_\ast, x_\ast)$

Além valores de inicialização comuns são, $\sigma^2_f = \mathbb{V}[y]$, $w^2_d = \frac{1}{\mathbb{V}[X_{:d}]}$ e $\sigma^2_y = 0.01 \sigma^2_f$ 


In [ ]:
class RegGP:
    def __init__(self):
        pass

    def estimate(self, X_train):
        pass

    def predict(self):
        pass